# Кольца Артем М.19-04
## Task 1
### Датасет — Facebook Comment Volume Dataset.

Предсказать, сколько комментариев наберёт пост. Задача предполагает реализацию градиентного спуска и подсчёта метрик оценки качества модели.

Этапы решения:

— нормировка значений фичей;

— кросс-валидация по пяти фолдам и обучение линейной регрессии;

— подсчёт R^2 (коэффициента детерминации) и RMSE.

In [382]:
import pandas as pd
import numpy as np

In [383]:
df = pd.read_csv('Features_Variant_1.csv', header=None)

In [384]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,50,51,52,53
0,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,...,0,0,0,0,0,0,0,0,1,0
1,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,...,0,0,0,0,0,0,0,1,0,0
2,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,...,1,0,0,0,0,0,0,0,1,0
3,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,...,1,0,0,1,0,0,0,0,0,0
4,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,...,0,0,0,0,0,1,0,0,0,0


Перемешаем данные сохраняя индексы.

In [385]:
df = df.sample(frac=1).reset_index(drop=True)

In [386]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,50,51,52,53
0,7564986,0,123241,12,0.0,35.0,3.365714,1.0,5.335913,0.0,...,1,0,0,1,0,0,0,0,0,0
1,329587,0,11271,4,0.0,97.0,12.101190,7.0,15.935749,0.0,...,0,1,0,1,0,0,0,0,0,4
2,9774,0,2397,16,0.0,57.0,2.609589,1.0,6.082057,0.0,...,1,0,1,0,0,0,0,0,0,0
3,700738,0,1600,34,0.0,64.0,5.703704,2.0,10.843379,0.0,...,0,0,0,0,0,0,0,0,1,0
4,158159,18,237,13,0.0,6.0,2.000000,1.0,1.885618,0.0,...,1,0,1,0,0,0,0,0,0,0


Создадим отдельный массив с целевым признаком y

In [387]:
y = df[53].values

In [388]:
y

array([0, 4, 0, ..., 0, 0, 0], dtype=int64)

In [389]:
y.shape

(40949,)

Остальные признаки в матрицу Х


In [390]:
X = df.iloc[:, 0:53].values

In [391]:
X

array([[7.564986e+06, 0.000000e+00, 1.232410e+05, ..., 0.000000e+00,
        0.000000e+00, 0.000000e+00],
       [3.295870e+05, 0.000000e+00, 1.127100e+04, ..., 0.000000e+00,
        0.000000e+00, 0.000000e+00],
       [9.774000e+03, 0.000000e+00, 2.397000e+03, ..., 0.000000e+00,
        0.000000e+00, 0.000000e+00],
       ...,
       [4.526533e+06, 0.000000e+00, 1.111340e+05, ..., 0.000000e+00,
        0.000000e+00, 0.000000e+00],
       [1.003900e+04, 4.000000e+00, 1.151000e+03, ..., 1.000000e+00,
        0.000000e+00, 0.000000e+00],
       [9.422000e+03, 7.750000e+03, 1.890000e+02, ..., 0.000000e+00,
        1.000000e+00, 0.000000e+00]])

In [392]:
X.shape

(40949, 53)

Найдем среднее значение признаков и стандартное отклонение по каждой колонке матрицы с признаками Х.

mean - среднее значение признаков

std - стандартное отклонение

In [393]:
mean = X.mean(axis=0)
std = X.std(axis=0)

Найдем признаками у которых std=0, те признаки не влияют на целевой признак.

In [394]:
np.where(std==0)

(array([37], dtype=int64),)

In [395]:
X = np.delete(X, 37, axis=1)

In [396]:
X.shape

(40949, 52)

Пересчитаем заново mean и std

In [397]:
mean = X.mean(axis=0)
std = X.std(axis=0)

In [398]:
np.where(std==0)

(array([], dtype=int64),)

Сделаем нормировку значений признаков посредством отнятия среднего значения по колонке от каждого значения в этой колонке и всё поделим на std этой колонки.

In [399]:
def normalization_values(X):
    for i in range(0, X.shape[0]):
        for j in range(0, X.shape[1]):
            X[i][j] = (X[i][j] - mean[j])/std[j]
    return X

In [400]:
X = normalization_values(X)

In [401]:
X.shape

(40949, 52)

In [402]:
X

array([[ 0.92123155, -0.22707468,  0.70710377, ..., -0.42147708,
        -0.40996237, -0.41162705],
       [-0.14504491, -0.22707468, -0.30224929, ..., -0.42147708,
        -0.40996237, -0.41162705],
       [-0.19217556, -0.22707468, -0.38224392, ..., -0.42147708,
        -0.40996237, -0.41162705],
       ...,
       [ 0.47345653, -0.22707468,  0.59796527, ..., -0.42147708,
        -0.40996237, -0.41162705],
       [-0.19213651, -0.22688044, -0.39347598, ...,  2.37260824,
        -0.40996237, -0.41162705],
       [-0.19222744,  0.14926801, -0.40214792, ..., -0.42147708,
         2.43924827, -0.41162705]])

Добавим колонку из единиц к X для W0 линейной регресии

In [403]:
np.hstack((X, np.ones((X.shape[0], 1))))

array([[ 0.92123155, -0.22707468,  0.70710377, ..., -0.40996237,
        -0.41162705,  1.        ],
       [-0.14504491, -0.22707468, -0.30224929, ..., -0.40996237,
        -0.41162705,  1.        ],
       [-0.19217556, -0.22707468, -0.38224392, ..., -0.40996237,
        -0.41162705,  1.        ],
       ...,
       [ 0.47345653, -0.22707468,  0.59796527, ..., -0.40996237,
        -0.41162705,  1.        ],
       [-0.19213651, -0.22688044, -0.39347598, ..., -0.40996237,
        -0.41162705,  1.        ],
       [-0.19222744,  0.14926801, -0.40214792, ...,  2.43924827,
        -0.41162705,  1.        ]])

In [404]:
X = np.hstack((X, np.ones((X.shape[0], 1))))

Функция прогноза модели



In [405]:
X.shape

(40949, 53)

In [406]:
def predict(X, w):
    return np.dot(X, w)

Градиент функции потери

In [407]:
def gradient_loss(X, y, w):
    return  np.dot(2/len(y) * X.transpose(), np.add(predict(X, w), - y))

Формула градиентного спуска:
![aльт](https://render.githubusercontent.com/render/math?math=%5CLarge%20w%5Et%20%3D%20w%5E%7Bt-1%7D%20-%20lr%2Agradient%20step&mode=display"a")
lr - шаг





In [419]:
def grad_des(X, y, w, lr, epochs_n):
    
    # Соединим признаки с целевым признаком
    dataf = np.concatenate((X, y.reshape(len(y), 1)), axis=1)
    
    for i in range(epochs_n):
        # Мешаем данные
        np.random.shuffle(dataf)
        # Обратно разделим признаки от целевого признака
        X = np.delete(dataf, 53, axis=1)
        y = np.delete(dataf, range(53), axis=1).transpose()[0]
        
        # Пропустим датасет через батчи
        batch_size = 1000
        
        for batch_n in range(int(len(y) / batch_size) + 1):
            start_n = batch_n * batch_size
            end_n = start_n + batch_size
            batch = X[start_n:end_n]
            y_batch = y[start_n:end_n]
            grad = gradient_loss(batch, y_batch, w)
            w = w - lr * grad
    
    return w

Поделим датасет на 5 частей для кросс-валидации

In [420]:
number_parts = 5

In [421]:
np.array_split(X, number_parts)

[array([[ 0.92123155, -0.22707468,  0.70710377, ..., -0.40996237,
         -0.41162705,  1.        ],
        [-0.14504491, -0.22707468, -0.30224929, ..., -0.40996237,
         -0.41162705,  1.        ],
        [-0.19217556, -0.22707468, -0.38224392, ..., -0.40996237,
         -0.41162705,  1.        ],
        ...,
        [ 0.17049126, -0.22532651,  2.04335848, ..., -0.40996237,
         -0.41162705,  1.        ],
        [-0.03452493, -0.22707468,  0.19497225, ...,  2.43924827,
         -0.41162705,  1.        ],
        [-0.17878691, -0.22702612, -0.39231311, ..., -0.40996237,
         -0.41162705,  1.        ]]),
 array([[-0.19166758, -0.22707468, -0.40377955, ..., -0.40996237,
         -0.41162705,  1.        ],
        [-0.19297121, -0.22707468, -0.40385166, ..., -0.40996237,
         -0.41162705,  1.        ],
        [-0.1651906 , -0.22702612, -0.37999029, ..., -0.40996237,
         -0.41162705,  1.        ],
        ...,
        [-0.17656988, -0.22707468, -0.39713586, ..., -

In [422]:
X_split = np.array_split(X, number_parts)

In [423]:
np.array_split(y, number_parts)

[array([ 0,  4,  0, ...,  0, 48,  0], dtype=int64),
 array([0, 0, 0, ..., 0, 0, 5], dtype=int64),
 array([ 2,  1,  0, ...,  0,  2, 51], dtype=int64),
 array([0, 1, 0, ..., 3, 0, 1], dtype=int64),
 array([0, 1, 3, ..., 0, 0, 0], dtype=int64)]

In [424]:
y_split = np.array_split(y, number_parts)

Введем функции вычисления метрик (качества данной модели)

In [425]:
def mse(y, y_pred):
    return ((y_pred-y)**2).sum()/len(y)

In [426]:
def rmse(y, y_pred):
    return np.sqrt(mserror(y, y_pred))

In [427]:
def r2(X, y, w):
    return 1 - (sum((y - np.dot(X, w))**2) / sum((y - np.mean(y))**2))

In [428]:
def cross_validation(features, target, number_parts):
    metrics = []
    weights = []
    
    for num in range(number_parts):
        X_train = []
        y_train = []
        # На каждом шаге в цикле одна часть обучается, остальные тестовые
        for i in range(number_parts):
            if i != num:
                
                X_train.append(X_split[i])
                y_train.append(y_split[i])
            else:
                X_test = X_split[i]
                y_test = y_split[i]
                
        X_train = np.concatenate(X_train, axis=0)
        y_train = np.concatenate(y_train, axis=0)
        # Обучение
        pred_weights = grad_des(X_train, y_train, np.zeros(X.shape[1]), lr=0.01, epochs_n=10)
        # Расчет метрик
        RMSE_train = rmse(y_train, np.dot(X_train, pred_weights))
        RMSE_test = rmse(y_test, np.dot(X_test, pred_weights))
        R2_train = r2(X_train, y_train, pred_weights)
        R2_test = r2(X_test, y_test, pred_weights)
        # Объединение метрик
        metrics.append([RMSE_train, RMSE_test, R2_train, R2_test])
        weights.append(pred_weights)
    
    return metrics, weights

In [433]:
metrics, weights = cross_validation(X_split, y_split, number_parts)

In [440]:
metrics_mean = []
metrics_std = []

In [441]:
for i in range(4):
    total_list = []
    for metric in metrics:
        total_list.append(metric[i])
    metrics_mean.append(np.mean(total_list))
    metrics_std.append(np.std(total_list))

In [444]:
df_metrics = {
        'fold 1': metrics[0], 
        'fold 2': metrics[1], 
        'fold 3': metrics[2], 
        'fold 4': metrics[3], 
        'fold 5': metrics[4],
        'mean': metrics_mean,
        'std': metrics_std
       }

In [445]:
df_metrics = pd.DataFrame(df_metrics, index=['RMSE_train', 'RMSE_test', 'R2_train', 'R2_test'])

In [446]:
weights_mean = []
weights_std = []

for i in range(53):
    total_list = []
    for weight_list in weights:
        total_list.append(weight_list[i])
    weights_mean.append(np.mean(total_list))
    weights_std.append(np.std(total_list))

In [447]:
data_weights = {
    'fold 1': weights[0], 
    'fold 2': weights[1], 
    'fold 3': weights[2], 
    'fold 4': weights[3], 
    'fold 5': weights[4],
    'mean': weights_mean,
    'std': weights_std
}

In [448]:
indexes = []
for i in range(len(weights[0])):
    indexes.append(f'w{i}')
    
df_weights = pd.DataFrame(data_weights, index=indexes)

In [451]:
result_df = pd.concat([df_metrics, df_weights])

In [452]:
result_df

,fold 1,fold 2,fold 3,fold 4,fold 5,mean,std
RMSE_train,29.900937,29.113005,28.409791,29.723229,29.304375,29.290267,0.522921
RMSE_test,26.795212,30.144119,33.214382,27.676482,29.542253,29.474490,2.229401
R2_train,0.307016,0.324667,0.323622,0.319094,0.320146,0.318909,0.006299
R2_test,0.369344,0.289557,0.276801,0.309295,0.299896,0.308978,0.032060
w0,0.320150,0.043162,-0.971348,0.142072,0.419557,-0.009281,0.498730
w1,-0.709293,-0.547359,-0.613828,-0.604503,-0.562444,-0.607486,0.056671
w2,-1.514646,-1.363356,-0.607928,-1.454200,-1.463283,-1.280683,0.339889
w3,-0.044454,-0.021954,-0.061957,-0.048313,-0.006415,-0.036618,0.019837
w4,-0.147970,-0.495346,-0.172382,0.144158,0.194170,-0.095474,0.248952
w5,0.357345,0.391040,0.480626,0.538591,0.291804,0.411881,0.087873
